<a href="https://colab.research.google.com/github/Pedrofelipe3349/Pedrofelipe3349/blob/main/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar a biblioteca datasets
!pip install datasets

In [ ]:
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from datasets import load_dataset

def baixar_recursos_nltk():
    nltk.download('punkt')
    nltk.download('stopwords')

def carregar_e_preparar_dados(nome_dataset):
    dataset = load_dataset(nome_dataset)
    labels = [
        "health", "ideology", "insult", "lgbtqphobia", "other_lifestyle",
        "physical_aspects", "profanity_obscene", "racism", "religious_intolerance", "sexism", "xenophobia"
    ]
    return dataset, labels

def ajustar_rotulos(data_split, labels):
    rotulos_ajustados = [
        [1 if instance['is_offensive'] == 'OFF' else 0] +
        [1 if instance[label] else 0 for label in labels]
        for instance in data_split
    ]
    return np.array(rotulos_ajustados)

def preprocessar_textos(textos):
    conjunto_stopwords = set(stopwords.words('portuguese'))
    textos_processados = [
        ' '.join([palavra for palavra in word_tokenize(texto, language='portuguese')
                  if palavra.lower() not in conjunto_stopwords and palavra.isalpha()])
        for texto in textos
    ]
    return textos_processados

def vetorizar_textos(textos):
    vetorizador = TfidfVectorizer()
    matriz_tfidf = vetorizador.fit_transform(textos).toarray()
    return matriz_tfidf, vetorizador

def construir_e_compilar_modelo(dim_input, num_classes):
    modelo = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(dim_input,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes + 1, activation='sigmoid')
    ])
    modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return modelo

def prever_sentencas_customizadas(modelo, sentencas, vetorizador, labels):
    preprocessadas = preprocessar_textos(sentencas)
    tfidf = vetorizador.transform(preprocessadas).toarray()
    probabilidades_predicao = modelo.predict(tfidf)
    return interpretar_predicoes(probabilidades_predicao, labels)

def interpretar_predicoes(probabilidades_predicao, labels, limiar_ofensivo=0.7, limiar_label=0.5):
    resultados = []
    for prob in probabilidades_predicao:
        if prob[0] > limiar_ofensivo:
            labels_ofensividade = [
                labels[i] for i, val in enumerate(prob[1:]) if val > limiar_label
            ] or ["Nenhuma categoria de ofensividade específica encontrada"]
            resultados.append(("OFF", labels_ofensividade))
        else:
            resultados.append(("NOT", None))
    return resultados

def main():
    baixar_recursos_nltk()
    dataset, labels = carregar_e_preparar_dados("dougtrajano/olid-br")
    rotulos_treino = ajustar_rotulos(dataset['train'], labels)
    rotulos_teste = ajustar_rotulos(dataset['test'], labels)

    textos_treino = preprocessar_textos(dataset['train']['text'])
    textos_teste = preprocessar_textos(dataset['test']['text'])

    tfidf_treino, vetorizador = vetorizar_textos(textos_treino)
    tfidf_teste = vetorizador.transform(textos_teste).toarray()

    modelo = construir_e_compilar_modelo(tfidf_treino.shape[1], len(labels))
    modelo.fit(tfidf_treino, rotulos_treino, epochs=10, batch_size=64, verbose=2)
    resultados = modelo.evaluate(tfidf_teste, rotulos_teste)
    print("Perda:", resultados[0], "Acurácia:", resultados[1])

    sentencas_customizadas = [
        "Viadinho!", "A planta floresceu!", "Olha que cara mais gordo, obeso.",
        "Estou muito feliz hoje.", "Mulher no volante, perigo constante", "Tenha um ótimo dia!",
        "Nossa, que leproso!", "Tinha que ser goiano, esse zé do mato.",
        "Seu imbecil!", "Hoje o dia está ensolarado e cheio de pássaros.",
        "Até que essa preta é bonita.", "Não consegue promoção porque é mulher, vagabunda."
    ]

    predicoes = prever_sentencas_customizadas(modelo, sentencas_customizadas, vetorizador, labels)
    for sentenca, predicao in zip(sentencas_customizadas, predicoes):
        print(f"Sentença: '{sentenca}'")
        print("Classificação:", predicao[0])
        if predicao[1]:
            print("Categoria de ofensividade:", predicao[1])
        print("="*50)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/5214 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1738 [00:00<?, ? examples/s]

Epoch 1/10
82/82 - 7s - loss: 0.4000 - accuracy: 0.4494 - 7s/epoch - 88ms/step
Epoch 2/10
82/82 - 9s - loss: 0.2512 - accuracy: 0.6513 - 9s/epoch - 109ms/step
Epoch 3/10
82/82 - 5s - loss: 0.2158 - accuracy: 0.6901 - 5s/epoch - 63ms/step
Epoch 4/10
82/82 - 6s - loss: 0.1694 - accuracy: 0.7300 - 6s/epoch - 76ms/step
Epoch 5/10
82/82 - 4s - loss: 0.1307 - accuracy: 0.7557 - 4s/epoch - 53ms/step
Epoch 6/10
82/82 - 4s - loss: 0.1040 - accuracy: 0.7570 - 4s/epoch - 51ms/step
Epoch 7/10
82/82 - 6s - loss: 0.0821 - accuracy: 0.7459 - 6s/epoch - 78ms/step
Epoch 8/10
82/82 - 5s - loss: 0.0660 - accuracy: 0.7591 - 5s/epoch - 55ms/step
Epoch 9/10
82/82 - 5s - loss: 0.0548 - accuracy: 0.7428 - 5s/epoch - 57ms/step
Epoch 10/10
82/82 - 6s - loss: 0.0464 - accuracy: 0.7426 - 6s/epoch - 77ms/step
55/55 [==============================] - 1s 10ms/step - loss: 0.3342 - accuracy: 0.8567
Perda: 0.33415305614471436 Acurácia: 0.8567318916320801
1/1 [==============================] - 0s 119ms/step
Sentença: '